# Init

In [1]:
from __future__ import absolute_import, division, print_function

import logging
import sys
logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format='%(asctime)s %(name)s-%(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')
import os
import matplotlib.pyplot as plt
plt.style.use("seaborn-colorblind")
import numpy as np

plt.rcParams['figure.autolayout'] = True
plt.rcParams['font.family'] = 'sans-serif'
_blue = [50.0 / 256.0, 117.0 / 256.0, 220.0 / 256.0]
_gray = [33.0 / 256.0, 36.0 / 256.0, 50.0 / 256.0]

logger = logging.getLogger("accuracy_comp")

logger.info("Started")

2019-05-23 14:11:48 matplotlib-DEBUG: $HOME=/home/oliverfl
2019-05-23 14:11:48 matplotlib-DEBUG: matplotlib data path /home/oliverfl/anaconda3/envs/py2/lib/python2.7/site-packages/matplotlib/mpl-data
2019-05-23 14:11:48 matplotlib-DEBUG: loaded rc file /home/oliverfl/anaconda3/envs/py2/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc
2019-05-23 14:11:48 matplotlib-DEBUG: matplotlib version 2.2.3
2019-05-23 14:11:48 matplotlib-DEBUG: interactive is False
2019-05-23 14:11:48 matplotlib-DEBUG: platform is linux2
2019-05-23 14:11:48 matplotlib-DEBUG: loaded modules: ['_multiprocessing', 'IPython.core.error', 'ipython_genutils.py3compat', 'traitlets.config.sys', 'concurrent.futures.types', 'ipykernel.parentpoller', 'traitlets.config.decorator', 'ctypes.os', 'pexpect.select', 'runpy', 'gc', 'numpy.lib.numpy', 'pty', 'prompt_toolkit.terminal.vt100_output', 'prompt_toolkit.key_binding.six', 'dateutil.parser.dateutil', 'pygments.filters', 'IPython.utils.pprint', 'numpy.core.info', '

2019-05-23 14:11:48 matplotlib-DEBUG: CACHEDIR=/home/oliverfl/.cache/matplotlib
2019-05-23 14:11:48 matplotlib.font_manager-DEBUG: Using fontManager instance from /home/oliverfl/.cache/matplotlib/fontList.json
2019-05-23 14:11:49 matplotlib.backends-DEBUG: backend module://ipykernel.pylab.backend_inline version unknown
2019-05-23 14:11:49 accuracy_comp-INFO: Started
2019-05-23 14:11:49 matplotlib.backends-DEBUG: backend module://ipykernel.pylab.backend_inline version unknown


# Define methods

Methods considered:
 * **AUC** = Area under the ROC curve
 * **ranked_accuracy** = sort the residues according to their relevance and seek the bandgap. Score the True positives above the bandgap
 * **ignoring irrelevant** = $\sum_{x} / \sum_{\phi} $ where $x$ = relevance of truly relevant atoms and $\phi$ = relevance of all atoms
 * **MSE accuracy** = Based on the normalized Mean squared error (MSE) $1-\frac{|\bar{x}-\bar{\phi}|^2}{|\bar{x}|^2+|\bar{\phi}|^2}$ where $\bar{\phi}$ is the measured importance and $\bar{x}$ is the true importance

In [2]:
def compute_AUC(relevant_residues, importance):
    """
    Computes area under ROC
    """

    auc = 0
    nresidues = importance.shape[0]
    actives = np.chararray(nresidues)
    actives[:] = 'd'
    ind_a = relevant_residues
    actives[ind_a] = 'a'

    actives_len = len(ind_a)
    decoys_len = nresidues - actives_len

    ind_scores_sorted = np.argsort(-importance)
    actives_sorted = actives[ind_scores_sorted]

    tp = 0
    fp = 0
    tp_rate = []
    fp_rate = []
    for j in actives_sorted:
        if j == 'a':
            tp += 1
        else:
            fp += 1
        tp_rate.append(float(tp) / float(actives_len))
        fp_rate.append(float(fp) / float(decoys_len))

    for j in range(len(fp_rate) - 1):
        auc += (fp_rate[j + 1] - fp_rate[j]) * (tp_rate[j + 1] + tp_rate[j]) / 2

    return auc


def compute_ranked_accuracy(relevant_residues, importance):
    """
    Computes ( TP + TN )/( P + N )
    """
    nresidues = importance.shape[0]

    importance_SORTED = -np.sort(-importance)
    importance_SORTED_ind = np.argsort(-importance)

    difference = []
    for j in range(nresidues - 1):
        difference.append(importance_SORTED[j] - importance_SORTED[j + 1])
    difference_max = max(difference)
    difference_max_index = difference.index(difference_max)

    POSITIVE = importance_SORTED_ind[0:difference_max_index + 1]
    NEGATIVE = importance_SORTED_ind[difference_max_index + 1:]

    ind_a = relevant_residues

    TP = len(list(set(POSITIVE) & set(ind_a)))
    # FP = len(POSITIVE) - TP

    ind_d = [j for j in np.arange(0, nresidues, 1) if j not in ind_a]
    TN = len(list(set(NEGATIVE) & set(ind_d)))
    # FN = len(NEGATIVE) - TN

    accuracy = (TP + TN) / nresidues

    return accuracy

def compute_relevant_fraction(relevant_residues, importance):
    return importance[relevant_residues].sum()/importance.sum()

def compute_mse_accuracy(relevant_residues, importance):
    true_importance = np.zeros(importance.shape)
    #The code is written to handle weighted true importance (i.e. when not all residues are equally relevant as well)
    true_importance[relevant_residues] = 1
    norm = np.linalg.norm(importance - true_importance)
    return 1 - norm**2 / (np.linalg.norm(true_importance)**2 + np.linalg.norm(importance)**2)

def compute_accuracy(relevant_residues, importance, method):
    if method == 'ranked_accuray':
        return compute_ranked_accuracy(relevant_residues, importance)
    elif method == 'auc':
        return compute_AUC(relevant_residues, importance)
    elif method == 'ignoring irrelevant':
        return compute_relevant_fraction(relevant_residues, importance)
    elif method == 'finding all':
        return compute_mse_accuracy(relevant_residues, importance)
    else:
        raise Exception("Unknown method {}".format(method))
        
logger.debug("Done")

2019-05-23 14:11:49 accuracy_comp-DEBUG: Done


# Construct artifical importance profiles

In [23]:
class ImportanceProfile(object):
    """
    Container class for importance and true values
    """
    def __init__(self, importance, relevant_residues, name):
        self.importance = importance
        self.relevant_residues = relevant_residues
        self.name = name
        self.methods = [
                #'auc', 'ranked_accuray',
                'finding all',
                'ignoring irrelevant'
                       ]
        self.accuracy = None
        
        
    def compute_accuracy(self):
        accuracy = []
        for method in self.methods:
            a = compute_accuracy(self.relevant_residues, self.importance, method)
            accuracy.append(abs(a))
        self.accuracy = np.array(accuracy)
        return self
    
    def plot(self, ax):
        ax.plot(self.importance, color='black')
        accuracy_text = "; ".join("{}: {:.2f}".format(self.methods[i], self.accuracy[i]) for i in range(len(self.accuracy)))
        for h in self.relevant_residues:
            ax.axvline(h, linestyle='--', color=_blue, linewidth=2)
        ax.set_title("{}\n({})".format(self.name,accuracy_text))
        #plt.suptitle(accuracy_text,x=0.2, y=0, horizontalalignment='left')
        ax.set_xlabel("Residue #")
        ax.set_ylabel("Importance")
        return self


def skew(importance, relevant_residues):
    for i,r in enumerate(relevant_residues):
        importance[r] /=(i+1)
    return importance

def noisify(importance, strength):
    noise = np.random.rand(*importance.shape)*strength
    importance = importance + noise
    importance /= importance.max() #normalize
    return importance

def create_profiles():
    nresidues = 50
    relevant_residues = [10, 20, 30, 40]
    perfect_importance = np.zeros(nresidues)
    perfect_importance[relevant_residues] = 1

    #Perfect is what it is -> should give a score of 1
    perfect = ImportanceProfile(perfect_importance.copy(), relevant_residues, "Perfect")

    #perfect skewed has no noise but difference importance per residue
    perfect_skewed = ImportanceProfile(skew(perfect_importance.copy(), relevant_residues), relevant_residues, "Perfect skewed")

    #Good has some noise to unimportant residues
    good_importance = noisify(perfect_importance, 0.25)
    good = ImportanceProfile(good_importance.copy(), relevant_residues, "Good")

    #Good_skewed 
    good_skewed = ImportanceProfile(skew(good_importance.copy(), relevant_residues), relevant_residues, "Good skewed")

    #Noisy
    #noisy_importance = noisify(perfect_importance, 0.99)
    #noisy = ImportanceProfile(noisy_importance.copy(), relevant_residues, "Noisy")
    #noisy_skewed = ImportanceProfile(skew(noisy_importance.copy(), relevant_residues), relevant_residues, "Noisy skewed")

    #Nonsense
    nonsense = ImportanceProfile(noisify(np.zeros(nresidues), 1), relevant_residues, "Random")
    
    #Inverted
    inverted = ImportanceProfile(1 - perfect_importance, relevant_residues, "Inverted")
    
    profiles = [perfect, 
                perfect_skewed, 
                good, 
                good_skewed, 
                #noisy, 
                #noisy_skewed, 
                nonsense, 
                inverted]
    
    return profiles
logger.debug("Done")

2019-05-23 14:26:05 accuracy_comp-DEBUG: Done


# Compare accuracy scores for measured vs actual relevance profile

In [24]:
profiles = create_profiles()
fig, axs = plt.subplots(2,3, sharex=True, sharey=True, figsize=(3*4.4,2*3))
for idx, ax in enumerate(axs.flatten()):
    p = profiles[idx]
    accuracy = p.compute_accuracy()
    p.plot(ax)
#     logger.info("\n\t-------%s-------------\n\n\t%s\n", 
#                 p.name, 
#                 "\n\t".join(["{:.2f}:\t({})".format(a, m) for m,a in accuracy])
#                )    
fig.savefig("../output/accuracy_scores_comparison.svg")
plt.clf()
logger.debug("Done")

2019-05-23 14:26:07 accuracy_comp-DEBUG: Done


<Figure size 950.4x432 with 0 Axes>

## Get som statistics

Average this over several runs of random values

In [31]:
niterations = 20
results = []
for i in range(niterations):
    profiles = create_profiles()
    for j, p in enumerate(profiles):
        p.compute_accuracy()
        #averaging code written when we kept data in a different way - TODO make it more comprehensible
        if i == 0:
            acc = [(p.methods[idx], [p.accuracy[idx]]) for idx in range(len(p.methods))]
            results.append((p.name, acc))
        else:
            old_accuracy = results[j][1]
            for k in range(len(p.accuracy)):
                old_accuracy[k][1].append(p.accuracy[k])
        
text = ""
for name, accuracy  in results:
    logger.info("\n\t-------%s-------------\n\n\t%s\n", 
                name, 
                "\n\t".join(["{:.2f} +- {:.2f}:\t({})".format(np.mean(a), np.std(a), m) for m,a in accuracy])
                )    
logger.debug("Done with %s iterations", niterations)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Perfect-------------

	1.00 +- 0.00:	(finding all)
	1.00 +- 0.00:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Perfect skewed-------------

	0.77 +- 0.00:	(finding all)
	1.00 +- 0.00:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Good-------------

	0.92 +- 0.01:	(finding all)
	0.44 +- 0.02:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Good skewed-------------

	0.66 +- 0.01:	(finding all)
	0.29 +- 0.02:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Random-------------

	0.17 +- 0.06:	(finding all)
	0.07 +- 0.02:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-INFO: 
	-------Inverted-------------

	0.00 +- 0.00:	(finding all)
	0.00 +- 0.00:	(ignoring irrelevant)

2019-05-23 14:31:34 accuracy_comp-DEBUG: Done with 20 iterations
